In [ ]:
# Import cubeit functions

import pytest
import numpy as np
from cubeit import (
    QuantumState,
    quantumregister,
    get_state,
    measure,
    h, s, t, x, y, z,
    cnot, cnot_10, swap, cz,
)
from cubeit.visualization import *

#Import plotting and visualisation libraries

import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual, Text, SelectMultiple, Layout, VBox, HBox, Button
from IPython.display import display

In [29]:

#Function to initialize quantum state
def initialize_state(n):
    global N_qubits
    N_qubits = n
    state = quantumregister(N_qubits)
    return state

#Function to apply gates
def apply_gate(state, gate, qubits):
    if gate == 'H':
        state.h(qubits)
    elif gate == 'S':
        state.s(qubits)
    elif gate == 'T':
        state.t(qubits)
    elif gate == 'X':
        state.x(qubits)
    elif gate == 'Y':
        state.y(qubits)
    elif gate == 'Z':
        state.z(qubits)
    elif gate == 'CNOT':
        state.cnot(qubits[0], qubits[1])
    elif gate == 'SWAP':
        state.swap(qubits[0], qubits[1])
    elif gate == 'CZ':
        state.cz(qubits[0], qubits[1])
    return state

In [33]:
#Define register
N_qubits = input("Enter number of qubits:")
state = quantumregister(int(N_qubits))
print(f'{N_qubits} qubits initialised in |0...0> state.')

#Apply gates
gates = input("Enter gates, each with qubit:")
for gate_input in gates.split(';'):
   parts = gate_input[:-1].strip().split('(')
   try:
      parts[1] = int(parts[1])
   except:
      if ',' in parts[1]:
         parts[1] = list(map(int, parts[1].split(',')))
      else:
         print('Invalid qubit input.')
   state = apply_gate(state, parts[0], parts[1])
   print(f'Applied {parts[0]} on qubit {parts[1]}.')

2 qubits initialised in |0...0> state.
Applied H on qubit 1.
Applied S on qubit 0.


In [ ]:
#Measure